In [1]:
from itertools import combinations_with_replacement

import numpy as np
from numpy.linalg import lstsq
from scipy.integrate import solve_ivp
import pandas as pd

import matplotlib.pyplot as plt
plt.rcParams['font.size'] = 14

np.random.seed(123)

In [2]:
# Simulate the Lorenz System
tstart, tstop = 0, 50
dt = 0.001
timepoints = np.arange(tstart, tstop + dt, dt)

σ, ρ, β = 10, 28, 8/3
x_0 = (-8, 8, 27)

def lorenz(t, x_t, σ=σ, ρ=ρ, β=β):
    """Lorenz differential equation, returns LHS (derivatives) at a point `x_t`."""
    x, y, z = x_t
    return σ*(y - x), x*(ρ - z) - y, x*y - β*z

x = solve_ivp(lorenz, (tstart, tstop), x_0, t_eval=timepoints).y.T
dx_dt = np.array([lorenz(0, x_t) for x_t in x])

In [3]:
# Sparse Identification of Non-linear Dynamics (SINDy)

def pool_data(x, poly_order):
    """
    Build model library Θ
    
    Parameters
    ----------
    x : ndarray like
        Observed dynamics of system (npoints, nvars).
    poly_order : int
        Maximum order of polynomial terms to construct.
   
    Returns
    -------
    ndarray like
        Library (npoints, nterms).
    """
    if poly_order < 1:
        raise ValueError('poly_order must be at least one')
    
    # Column for the constant term
    columns = [np.ones_like(x[:, 0])]

    # Products of r columns (including each column on its own for linear terms) 
    for r in range(1, poly_order+1):
        columns += [np.prod(columnset, axis=0) for columnset in combinations_with_replacement(x.T, r=r)]

    return np.column_stack(columns)


def sparsify_dynamics(Θ, dx_dt, λ, niters=10):
    """
    Sequential Thresholded Least-squares
    
    Parameters
    ----------
    Θ : ndarray like
        Library of non-linear functions (npoints, nterms).
    dx_dt : ndarray like
        Observed derivatives (LHS) of system (npoints, nvars).        
    λ : float
        Threshold for sparsification-regularization.
    niters : int
        Number of times to apply sparsification step.

    Returns
    -------
    ndarray like
        Coefficients of terms (nterms, nvars).

    See Also
    --------
        Brunton etal, Discovering governing equations from data ... (https://doi.org/10.1073/pnas.1517384113)
    """
    # Initial guess: Least-squares
    Ξ, *_ = lstsq(Θ, dx_dt, rcond=None)

    for _ in range(niters):
        # Threshold - set small coefficients to zero
        small_terms = abs(Ξ) < λ
        Ξ[small_terms] = 0
        large_terms = ~small_terms
        for dim in range(dx_dt.shape[1]):
            # Regress dynamics onto surviving terms to find a sparse Ξ
            terms = large_terms[:, dim]
            Ξ[terms, dim], *_ = lstsq(Θ[:, terms], dx_dt[:, dim], rcond=None)

    return Ξ

In [4]:
# Maximum polynomial order of non-linear terms
poly_order = 3

# Sparsification knob λ
λ = 0.025

# Generate the library Θ of candidate non-linear functions of coordinates 
Θ = pool_data(x, poly_order=poly_order)

# Compute the sparse regression Ξ fromn the library and the derivatives
Ξ = sparsify_dynamics(Θ, dx_dt, λ=λ)

In [5]:
def term_names(variable_names, poly_order):
    terms = ['1']
    for r in range(1, poly_order+1):
        terms += [''.join(term) for term in combinations_with_replacement(variable_names, r=r)]
    return terms

variable_names = list('xyz')
library_names = term_names(variable_names, poly_order)
derivative_names = [f'{name}_dot' for name in variable_names]
with pd.option_context('display.precision', 2, 'display.width', 5):
    display(pd.DataFrame(Ξ, index=library_names, columns=derivative_names).T)

,1,x,y,z,xx,xy,xz,yy,yz,zz,xxx,xxy,xxz,xyy,xyz,xzz,yyy,yyz,yzz,zzz
x_dot,0.0,-10.0,10.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
y_dot,0.0,28.0,-1.0,0.00,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
z_dot,0.0,0.0,0.0,-2.67,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
